In [29]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import tqdm.notebook as tqdm
import datetime

In [30]:
now = datetime.datetime.now()

In [31]:
def createURL(year, month, day):
    url = 'https://keirin.kdreams.jp/racecard/' + str(year) + "/" + str(month).zfill(2) + '/' + str(day).zfill(2) + '/'
    return url

In [32]:
source_url = [createURL(now.year, now.month, now.day)]

In [35]:
def get_race_id(sourceURLs):
    race_urls = {}
    for sourceURL in tqdm.tqdm(sourceURLs):
        try:
            req = requests.get(sourceURL)
            soup = BeautifulSoup(req.content, 'html.parser')
            time.sleep(1)
            race_html = soup.find_all('a', class_='JS_POST_THROW')
            for html in race_html:
                url = html.get('href')
                if 'racedetail' in url:
                    race_id = re.sub(r'\D', '', url)
                    race_urls[race_id] = url
        except:
            print("error")
            break
    return race_urls

In [40]:
race_urls = get_race_id(source_url)

In [38]:
main_colum = ['予想', '好気合', '総評', '枠番', '車番', '選手名府県/年齢/期別', '級班', '脚質', 'ギヤ倍数', '競走得点', '1着', '2着', '3着', '着外']

In [45]:
def scrape_race_data(race_urls):
    for race_id, url in tqdm.tqdm(race_urls.items()):
        race_data = {}
        try:
            main = pd.read_html(url)
            df = main[4][:-1]
            df.columns = main_colum
            df = df.astype(str)
            race_data[race_id] = df
            time.sleep(1)
        except IndexError:
            print('IndexError: {}', url)
            continue
        except KeyError:
            print('keyerror: {}', url)
            continue
        except ValueError:
            print("ValueError: {}", url)
            continue
        except :
            traceback.print_exc()
            break
    return race_data

In [46]:
race_data = scrape_race_data(race_urls)

In [47]:
for key in race_data.keys():
    race_data[key].index = [key]*len(race_data[key])
race_data = pd.concat([race_data[key] for key in race_data.keys()], sort=False)

In [49]:
race_data.to_pickle("data/race_data_ongoing.pkl")